In [1]:
x = 3.14559
print(round(x, 2))  
print(round(x, 3))  


3.15
3.146


# Ghanem

In [3]:
# fetch_exchange_range.py
import os
import re
import time
import json
import csv
import socket
from datetime import date, timedelta
from typing import Iterable, Dict, Any, List, Optional, Tuple

import requests
from requests.exceptions import ConnectTimeout, ReadTimeout, ProxyError, SSLError, ConnectionError as ReqConnectionError

# =========================
# CONFIG
# =========================
BASE_URL = "http://192.168.0.2/Production_ExchangeRatesAPI/ExchangeRates/getExchangeRates"
BASE_SAVE_DIR = "WebService/data"

# Timeouts: (connect_timeout, read_timeout)
CONNECT_TIMEOUT = 5
READ_TIMEOUT = 30
MAX_RETRIES = 3
RETRY_BACKOFF_BASE = 2

DATE_RE_MMDDYYYY = re.compile(r"^\d{2}/\d{2}/\d{4}$")
NO_PROXY = {"http": None, "https": None}

# =========================
# HELPERS
# =========================
def validate_mmddyyyy(s: str) -> None:
    if not DATE_RE_MMDDYYYY.match(s or ""):
        raise ValueError(f"Date must be MM/DD/YYYY, got {s}")

def mmddyyyy(d: date) -> str:
    return f"{d.month:02d}/{d.day:02d}/{d.year:04d}"

def daterange_days(start: date, end: date) -> Iterable[date]:
    d = start
    while d <= end:
        yield d
        d += timedelta(days=1)

def ensure_dir(p: str) -> None:
    if not os.path.isdir(p):
        os.makedirs(p, exist_ok=True)

def _tcp_preflight(host: str, port: int = 80, timeout: int = 3) -> bool:
    """Quick TCP connect preflight—returns True if TCP handshake succeeds."""
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except OSError:
        return False

def _make_session(disable_env_proxies: bool = True, force_no_proxy: bool = True) -> requests.Session:
    s = requests.Session()
    s.trust_env = not disable_env_proxies  # False → do NOT inherit env proxies
    if force_no_proxy:
        s.proxies.update(NO_PROXY)
    s.headers.update({"Content-Type": "application/json", "Accept": "application/json"})
    return s

# ---------- shaping ----------
def flatten_rows(obj: Any) -> List[Dict[str, Any]]:
    """
    Normalize unknown API shape to a list[dict] for saving CSV/JSON.
    """
    if isinstance(obj, list):
        return [x if isinstance(x, dict) else {"value": x} for x in obj]
    if isinstance(obj, dict):
        if isinstance(obj.get("data"), list):
            return [x if isinstance(x, dict) else {"value": x} for x in obj["data"]]
        return [obj]
    return [{"value": obj}]

# ---------- filters (hard) ----------
_FRF = "FRF"

def _extract_pair_ci(row: Dict[str, Any]) -> Tuple[str, str]:
    if not isinstance(row, dict):
        return "", ""
    lm = {k.lower(): k for k in row.keys()}

    def _pick(keys: List[str]) -> str:
        for k in keys:
            real = lm.get(k)
            if real:
                v = row.get(real, "")
                return str(v).strip().upper() if isinstance(v, (str, int, float)) else ""
        return ""

    f = _pick(["fromcurrency", "sourcecurrency", "from", "basecurrency"])
    t = _pick(["tocurrency", "targetcurrency", "to", "quotecurrency", "targetcurrency"])
    return f, t

def _same_currency_pair(row: Dict[str, Any]) -> bool:
    f, t = _extract_pair_ci(row)
    return bool(f) and f == t

def _has_frf(row: Dict[str, Any]) -> bool:
    f, t = _extract_pair_ci(row)
    return f == _FRF or t == _FRF

def _filter_rows_hard(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Remove rows where From==To or either side is FRF.
    """
    out = []
    for r in rows:
        if not isinstance(r, dict):
            continue
        if _same_currency_pair(r):
            continue
        if _has_frf(r):
            continue
        out.append(r)
    return out

# ---------- IO ----------
def _save_day_json(out_dir: str, d: date, rows: List[Dict[str, Any]]) -> str:
    json_dir = os.path.join(out_dir, "exchange_rates_json")
    ensure_dir(json_dir)
    p = os.path.join(json_dir, f"{d.isoformat()}.json")
    # Save under {"data": [...] } like your Fixer script
    with open(p, "w", encoding="utf-8") as f:
        json.dump({"data": rows}, f, ensure_ascii=False, indent=2)
    return p

def _append_csv(out_dir: str, rows: List[Dict[str, Any]]) -> None:
    if not rows:
        return
    csv_path = os.path.join(out_dir, "exchange_rates_agg.csv")

    # discover headers from existing + new
    all_keys = set()
    if os.path.exists(csv_path):
        with open(csv_path, "r", newline="", encoding="utf-8") as f:
            rdr = csv.DictReader(f)
            all_keys.update(rdr.fieldnames or [])
    for r in rows:
        all_keys.update(r.keys())
    fieldnames = sorted(all_keys)

    # load existing
    existing: List[Dict[str, Any]] = []
    if os.path.exists(csv_path):
        with open(csv_path, "r", newline="", encoding="utf-8") as f:
            rdr = csv.DictReader(f)
            existing = list(rdr)

    ensure_dir(out_dir)
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in existing:
            w.writerow({k: r.get(k, "") for k in fieldnames})
        for r in rows:
            w.writerow({k: r.get(k, "") for k in fieldnames})

# ---------- HTTP ----------
def _post_one_day(d: date, session: requests.Session) -> Any:
    payload = {"FromDate": mmddyyyy(d), "ToDate": mmddyyyy(d)}
    validate_mmddyyyy(payload["FromDate"]); validate_mmddyyyy(payload["ToDate"])

    last_err: Optional[Exception] = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f"→ POST {BASE_URL} body={payload} (attempt {attempt}/{MAX_RETRIES})")
            r = session.post(
                BASE_URL,
                json=payload,
                timeout=(CONNECT_TIMEOUT, READ_TIMEOUT),
                allow_redirects=False,
            )
            print(f"   HTTP {r.status_code}")
            r.raise_for_status()
            try:
                return r.json()
            except ValueError:
                return {"raw": r.text}
        except (ConnectTimeout, ReadTimeout) as e:
            last_err = e
            print(f"   Timeout: {e}")
        except (ProxyError, SSLError, ReqConnectionError, requests.HTTPError) as e:
            last_err = e
            print(f"   Connection/HTTP error: {e}")
        except Exception as e:
            last_err = e
            print(f"   Other error: {e}")

        time.sleep(min(RETRY_BACKOFF_BASE ** attempt, 8))

    raise RuntimeError(f"[{d}] Failed after {MAX_RETRIES} attempts: {last_err}")

# ---------- main ----------
def fetch_exchange_range(start: date, end: date) -> None:
    # Preflight
    host = BASE_URL.split("://", 1)[1].split("/", 1)[0]
    if not _tcp_preflight(host, 80, timeout=3):
        raise RuntimeError(
            f"Cannot open TCP to {host}:80. Likely VPN route/firewall issue. "
            "Check that 192.168.0.0/24 is routed via VPN and outbound :80 is allowed."
        )

    session = _make_session(disable_env_proxies=True, force_no_proxy=True)

    for d in daterange_days(start, end):
        day_dir = os.path.join(BASE_SAVE_DIR, d.isoformat())
        ensure_dir(day_dir)
        print(f"[fetch] {d.isoformat()} → {day_dir}")

        obj = _post_one_day(d, session)

        # Normalize to list[dict] and apply hard filters
        rows = flatten_rows(obj)
        rows = _filter_rows_hard(rows)

        # Optional stamp (helps later if CSV is used as source)
        for r in rows:
            if isinstance(r, dict):
                r.setdefault("_service_date", d.isoformat())

        _save_day_json(day_dir, d, rows)
        _append_csv(day_dir, rows)

    print("[done] fetch_exchange_range finished.")

# === RUN EXAMPLE ===
if __name__ == "__main__":
    # set your desired window here:
    fetch_exchange_range(date(2025, 7, 1), date(2025, 7, 31))


[fetch] 2025-07-01 → WebService/data\2025-07-01
→ POST http://192.168.0.2/Production_ExchangeRatesAPI/ExchangeRates/getExchangeRates body={'FromDate': '07/01/2025', 'ToDate': '07/01/2025'} (attempt 1/3)
   HTTP 200
[fetch] 2025-07-02 → WebService/data\2025-07-02
→ POST http://192.168.0.2/Production_ExchangeRatesAPI/ExchangeRates/getExchangeRates body={'FromDate': '07/02/2025', 'ToDate': '07/02/2025'} (attempt 1/3)
   HTTP 200
[fetch] 2025-07-03 → WebService/data\2025-07-03
→ POST http://192.168.0.2/Production_ExchangeRatesAPI/ExchangeRates/getExchangeRates body={'FromDate': '07/03/2025', 'ToDate': '07/03/2025'} (attempt 1/3)
   HTTP 200
[fetch] 2025-07-04 → WebService/data\2025-07-04
→ POST http://192.168.0.2/Production_ExchangeRatesAPI/ExchangeRates/getExchangeRates body={'FromDate': '07/04/2025', 'ToDate': '07/04/2025'} (attempt 1/3)
   HTTP 200
[fetch] 2025-07-05 → WebService/data\2025-07-05
→ POST http://192.168.0.2/Production_ExchangeRatesAPI/ExchangeRates/getExchangeRates body={'

In [5]:
import os
import re
import json
import csv
from typing import Dict, Any, List, Tuple, Set

BASE_DIR = "WebService/data"

# ✱ FIX: point to WebService/needed_rates.txt, not WebService/data/needed_rates.txt
# You can override with env NEEDED_RATES_FILE if you want a different path.
NEEDED_FILE = os.environ.get("NEEDED_RATES_FILE", "WebService/needed_rates.txt")

# match single-day directories: YYYY-MM-DD
RUN_DIR_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}$")

def ensure_dir(p: str) -> None:
    if not os.path.isdir(p):
        os.makedirs(p, exist_ok=True)

def _norm_currency(s: str) -> str:
    return (s or "").strip().upper()

def _find_key_case_insensitive(d: Dict[str, Any], *candidates: str) -> str:
    lower_map = {k.lower(): k for k in d.keys()}
    for c in candidates:
        k = lower_map.get(c.lower())
        if k is not None:
            return k
    return ""

def _row_pair(row: Dict[str, Any]) -> Tuple[str, str]:
    if not isinstance(row, dict):
        return "", ""
    from_k = _find_key_case_insensitive(row, "FromCurrency", "From", "BaseCurrency", "SourceCurrency")
    to_k   = _find_key_case_insensitive(row, "ToCurrency", "To", "QuoteCurrency", "TargetCurrency")
    f = _norm_currency(row.get(from_k, "")) if from_k else ""
    t = _norm_currency(row.get(to_k, ""))   if to_k else ""
    return f, t

def load_needed_pairs(path: str) -> Set[Tuple[str, str]]:
    pairs: Set[Tuple[str, str]] = set()
    if not os.path.isfile(path):
        raise SystemExit(
            f"[FATAL] needed list not found: {path}\n"
            "Create it (one pair per line like 'AED/USD') or set NEEDED_RATES_FILE env var."
        )
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s or s.startswith("#"):
                continue
            parts = s.split("/") if "/" in s else s.replace(" ", "/").split("/")
            if len(parts) != 2:
                continue
            a, b = _norm_currency(parts[0]), _norm_currency(parts[1])
            if a and b:
                pairs.add((a, b))
    if not pairs:
        raise SystemExit(f"[FATAL] needed list is empty: {path}")
    print(f"[needed] {len(pairs)} pairs loaded from {path}")
    return pairs

def should_keep(row: Dict[str, Any], include_set: Set[Tuple[str, str]]) -> bool:
    # include_set is guaranteed non-empty (see load_needed_pairs)
    f, t = _row_pair(row)
    if not f or not t:
        return False
    return (f, t) in include_set

def filter_json_object(obj: Any, include_set: Set[Tuple[str, str]]) -> Any:
    try:
        if isinstance(obj, list):
            return [r for r in obj if isinstance(r, dict) and should_keep(r, include_set)]
        if isinstance(obj, dict):
            if "data" in obj and isinstance(obj["data"], list):
                new_obj = dict(obj)
                new_obj["data"] = [r for r in obj["data"] if isinstance(r, dict) and should_keep(r, include_set)]
                return new_obj
            return {"data": [obj]} if should_keep(obj, include_set) else {"data": []}
        return {"data": []}
    except Exception:
        return {"data": []}

def filter_all_jsons(run_dir: str, include_set: Set[Tuple[str, str]]) -> Tuple[int, int]:
    json_in = os.path.join(run_dir, "exchange_rates_json")
    json_out = os.path.join(run_dir, "exchange_rates_json_filtered")
    ensure_dir(json_out)

    processed = 0
    written = 0

    if not os.path.isdir(json_in):
        print(f"[warn] JSON input dir not found: {json_in}")
        return (0, 0)

    for name in os.listdir(json_in):
        if not name.lower().endswith(".json"):
            continue
        inp = os.path.join(json_in, name)
        out = os.path.join(json_out, name)
        try:
            with open(inp, "r", encoding="utf-8") as f:
                obj = json.load(f)
            new_obj = filter_json_object(obj, include_set)
            with open(out, "w", encoding="utf-8") as f:
                json.dump(new_obj, f, ensure_ascii=False, indent=2)
            processed += 1
            written += 1
        except Exception as e:
            print(f"[json] skip {inp}: {e}")
            processed += 1
    return processed, written

def filter_csv(run_dir: str, include_set: Set[Tuple[str, str]]) -> Tuple[int, int]:
    csv_in = os.path.join(run_dir, "exchange_rates_agg.csv")
    csv_out = os.path.join(run_dir, "exchange_rates_agg.filtered.csv")

    if not os.path.isfile(csv_in):
        print(f"[warn] CSV input not found: {csv_in}")
        return (0, 0)

    with open(csv_in, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        fieldnames = rdr.fieldnames or []
        rows = list(rdr)

    if not fieldnames:
        print(f"[warn] CSV has no header: {csv_in}")
        return (0, 0)

    kept: List[Dict[str, Any]] = []
    for r in rows:
        if should_keep(r, include_set):
            kept.append(r)

    if "FromCurrency" not in fieldnames:
        fieldnames = fieldnames + ["FromCurrency"]
    if "ToCurrency" not in fieldnames:
        fieldnames = fieldnames + ["ToCurrency"]

    with open(csv_out, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in kept:
            out_row = {k: r.get(k, "") for k in fieldnames}
            fcur, tcur = _row_pair(r)
            if fcur and not out_row.get("FromCurrency"):
                out_row["FromCurrency"] = fcur
            if tcur and not out_row.get("ToCurrency"):
                out_row["ToCurrency"] = tcur
            w.writerow(out_row)

    return (len(rows), len(kept))

def list_all_day_dirs(base_dir: str) -> List[str]:
    if not os.path.isdir(base_dir):
        return []
    out: List[str] = []
    for name in os.listdir(base_dir):
        p = os.path.join(base_dir, name)
        if os.path.isdir(p) and RUN_DIR_PATTERN.match(name):
            out.append(p)
    out.sort()
    return out

# === RUN ===
if __name__ == "__main__":
    day_dirs = list_all_day_dirs(BASE_DIR)
    if not day_dirs:
        raise SystemExit(f"No day directories found under {BASE_DIR} matching YYYY-MM-DD")

    needed = load_needed_pairs(NEEDED_FILE)  # ✱ now fatal if missing/empty
    total_json_in = total_json_out = total_csv_in = total_csv_out = 0

    for run_dir in day_dirs:
        print(f"\n[run-dir] {run_dir}")
        j_in, j_out = filter_all_jsons(run_dir, needed)
        print(f"[done] JSON files processed={j_in}, written={j_out} → {os.path.join(run_dir, 'exchange_rates_json_filtered')}")
        c_in, c_out = filter_csv(run_dir, needed)
        print(f"[done] CSV rows in={c_in}, out={c_out} → {os.path.join(run_dir, 'exchange_rates_agg.filtered.csv')}")
        total_json_in += j_in; total_json_out += j_out
        total_csv_in  += c_in; total_csv_out  += c_out

    print(f"\n[summary] JSON in={total_json_in}, JSON out={total_json_out}; CSV in={total_csv_in}, CSV out={total_csv_out}")


[needed] 208 pairs loaded from WebService/needed_rates.txt

[run-dir] WebService/data\2025-07-01
[done] JSON files processed=1, written=1 → WebService/data\2025-07-01\exchange_rates_json_filtered
[done] CSV rows in=784, out=199 → WebService/data\2025-07-01\exchange_rates_agg.filtered.csv

[run-dir] WebService/data\2025-07-02
[done] JSON files processed=1, written=1 → WebService/data\2025-07-02\exchange_rates_json_filtered
[done] CSV rows in=784, out=199 → WebService/data\2025-07-02\exchange_rates_agg.filtered.csv

[run-dir] WebService/data\2025-07-03
[done] JSON files processed=1, written=1 → WebService/data\2025-07-03\exchange_rates_json_filtered
[done] CSV rows in=784, out=199 → WebService/data\2025-07-03\exchange_rates_agg.filtered.csv

[run-dir] WebService/data\2025-07-04
[done] JSON files processed=1, written=1 → WebService/data\2025-07-04\exchange_rates_json_filtered
[done] CSV rows in=784, out=199 → WebService/data\2025-07-04\exchange_rates_agg.filtered.csv

[run-dir] WebService

In [1]:
import os
import re
import csv
import json
from decimal import Decimal, ROUND_HALF_UP, InvalidOperation
from typing import Optional, List, Dict, Any, Tuple

BASE_DIR = "WebService/data"
CURRENCY_RULES_CSV = "WebService/Currency.csv"  # round_digits,target_currency
RUN_DIR_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}$")
FNAME_DATE_RE = re.compile(r"^(\d{4})-(\d{2})-(\d{2})\.json$", re.IGNORECASE)

def list_all_day_dirs(base_dir: str) -> List[str]:
    if not os.path.isdir(base_dir):
        return []
    out: List[str] = []
    for name in os.listdir(base_dir):
        p = os.path.join(base_dir, name)
        if os.path.isdir(p) and RUN_DIR_PATTERN.match(name):
            out.append(p)
    out.sort()
    return out

def _norm_ccy(x: Any) -> str:
    s = (x or "").strip().upper()
    return s if re.fullmatch(r"[A-Z]{3,4}", s) else ""

def _parse_rate(x: Any) -> Optional[float]:
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    sx = str(x).strip().replace(",", "")
    try:
        return float(sx)
    except ValueError:
        return None

def _is_mmddyyyy(s: str) -> bool:
    return bool(re.fullmatch(r"\d{2}/\d{2}/\d{4}", s or ""))

def _to_ddmmyyyy_from_iso(iso: str) -> str:
    m = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", iso or "")
    if not m:
        return ""
    yyyy, mm, dd = m.group(1), m.group(2), m.group(3)
    return f"{dd}.{mm}.{yyyy}"

def _mmddyyyy_to_ddmmyyyy_dots(s: str) -> str:
    m = re.fullmatch(r"(\d{2})/(\d{2})/(\d{4})", s or "")
    if not m:
        return ""
    mm, dd, yyyy = m.group(1), m.group(2), m.group(3)
    return f"{dd}.{mm}.{yyyy}"

def _key_ci(d: Dict[str, Any], *cands: str) -> Optional[str]:
    lm = {k.lower(): k for k in d.keys()}
    for c in cands:
        k = lm.get(c.lower())
        if k is not None:
            return k
    return None

def load_rounding_rules(csv_path: str) -> Dict[str, int]:
    rules: Dict[str, int] = {}
    if not os.path.isfile(csv_path):
        print(f"[warn] rounding rules CSV not found: {csv_path} (rates will not be rounded)")
        return rules
    with open(csv_path, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        for row in rdr:
            if not isinstance(row, dict):
                continue
            lower_map = {k.lower(): k for k in row.keys()}
            k_digits = lower_map.get("round_digits")
            k_ccy    = lower_map.get("target_currency")
            if not k_digits or not k_ccy:
                continue
            try:
                digits = int(str(row[k_digits]).strip())
            except (ValueError, TypeError):
                continue
            ccy = _norm_ccy(row[k_ccy])
            if not ccy:
                continue
            if digits < 0: digits = 0
            if digits > 12: digits = 12
            rules[ccy] = digits
    return rules

def _round_rate_half_up(rate: float, digits: int) -> float:
    try:
        q = Decimal("1").scaleb(-digits)
        d = Decimal(str(rate)).quantize(q, rounding=ROUND_HALF_UP)
        return float(d)
    except (InvalidOperation, ValueError):
        return rate

def apply_rounding(rows: List[Dict[str, Any]], rules: Dict[str, int]) -> List[Dict[str, Any]]:
    """
    Apply rounding per rules to rows *in-place*.
    NEW: If rounding would turn a non-zero rate into 0.0, keep the original rate
    and record the case in a zero-exception list (returned).
    Returns:
        zero_exceptions: list of dicts describing rows where rounding would yield zero.
    """
    zero_exceptions: List[Dict[str, Any]] = []
    if not rules:
        return zero_exceptions

    for r in rows:
        try:
            to_ccy = _norm_ccy(r.get("ToCurrency"))
            if not to_ccy:
                continue
            digits = rules.get(to_ccy)
            if digits is None:
                continue
            rate = r.get("ExchangeRate")
            if rate is None:
                continue
            rate_f = _parse_rate(rate)
            if rate_f is None:
                continue

            rounded = _round_rate_half_up(rate_f, digits)

            # If rounding makes a non-zero rate become zero, DON'T round.
            if rate_f != 0.0 and rounded == 0.0:
                zero_exceptions.append({
                    "FromCurrency": r.get("FromCurrency"),
                    "ToCurrency": r.get("ToCurrency"),
                    "ValidFrom": r.get("ValidFrom"),
                    "ExchangeRateType": r.get("ExchangeRateType"),
                    "Quotation": r.get("Quotation"),
                    "original_exchange_rate": rate_f,
                    "attempted_digits": digits,
                    "would_round_to": rounded
                })
                # keep original: do NOT assign rounded
                continue

            # Normal case: apply rounded value
            r["ExchangeRate"] = rounded

        except Exception:
            # Swallow per-row errors to avoid breaking the whole day; no logging change here.
            continue

    return zero_exceptions

def normalize_row_from_csv(raw: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    k_from = _key_ci(raw, "FromCurrency", "sourceCurrency", "From", "BaseCurrency", "SourceCurrency")
    k_to   = _key_ci(raw, "ToCurrency", "targetCurrency", "To", "QuoteCurrency", "TargetCurrency")
    fcur = _norm_ccy(raw.get(k_from, "")) if k_from else ""
    tcur = _norm_ccy(raw.get(k_to, ""))   if k_to else ""
    if not fcur or not tcur:
        return None

    k_rate = _key_ci(raw, "exChangeRateValue", "ExchangeRate", "Rate", "FxRate", "Value")
    rate = _parse_rate(raw.get(k_rate)) if k_rate else None
    if rate is None:
        return None

    k_date_csv = _key_ci(raw, "exChangeRateDate")
    valid_from = ""
    if k_date_csv:
        cand = (raw.get(k_date_csv) or "").strip()
        if _is_mmddyyyy(cand):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(cand)

    if not valid_from:
        k_iso = _key_ci(raw, "_service_date")
        if k_iso and raw.get(k_iso):
            valid_from = _to_ddmmyyyy_from_iso(str(raw[k_iso]).strip())

    if not valid_from:
        return None

    return {
        "ExchangeRateType": "M",
        "FromCurrency": fcur,
        "ToCurrency": tcur,
        "ValidFrom": valid_from,
        "Quotation": "Direct",
        "ExchangeRate": rate,
    }

def normalize_row_from_json(raw: Dict[str, Any], fallback_iso_date: str = "") -> Optional[Dict[str, Any]]:
    k_from = _key_ci(raw, "FromCurrency", "sourceCurrency", "From", "BaseCurrency", "SourceCurrency")
    k_to   = _key_ci(raw, "ToCurrency", "targetCurrency", "To", "QuoteCurrency", "TargetCurrency")
    fcur = _norm_ccy(raw.get(k_from, "")) if k_from else ""
    tcur = _norm_ccy(raw.get(k_to, ""))   if k_to else ""
    if not fcur or not tcur:
        return None

    k_rate = _key_ci(raw, "exChangeRateValue", "ExchangeRate", "Rate", "FxRate", "Value")
    rate = _parse_rate(raw.get(k_rate)) if k_rate else None
    if rate is None:
        return None

    k_mmdd = _key_ci(raw, "exChangeRateDate")
    k_other= _key_ci(raw, "ValidFrom", "Date", "RateDate", "_service_date")
    valid_from = ""
    if k_mmdd:
        cand = (raw.get(k_mmdd) or "").strip()
        if _is_mmddyyyy(cand):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(cand)

    if not valid_from and k_other and raw.get(k_other):
        s = str(raw[k_other]).strip()
        if _is_mmddyyyy(s):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(s)
        else:
            valid_from = _to_ddmmyyyy_from_iso(s)

    if not valid_from and fallback_iso_date:
        valid_from = _to_ddmmyyyy_from_iso(fallback_iso_date)

    if not valid_from:
        return None

    k_type = _key_ci(raw, "ExchangeRateType", "Type")
    k_quo  = _key_ci(raw, "Quotation", "QuoteType", "Side")
    etype = str(raw.get(k_type)).strip() if (k_type and raw.get(k_type)) else "M"
    quo   = str(raw.get(k_quo)).strip()  if (k_quo  and raw.get(k_quo))  else "Direct"

    return {
        "ExchangeRateType": etype,
        "FromCurrency": fcur,
        "ToCurrency": tcur,
        "ValidFrom": valid_from,
        "Quotation": quo,
        "ExchangeRate": rate,
    }

def load_from_filtered_csv(run_dir: str, skip_stats: Dict[str, int]) -> List[Dict[str, Any]]:
    path = os.path.join(run_dir, "exchange_rates_agg.filtered.csv")
    if not os.path.isfile(path):
        return []
    with open(path, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        rows_raw = list(rdr)

    out: List[Dict[str, Any]] = []
    for r in rows_raw:
        nr = normalize_row_from_csv(r)
        if nr:
            out.append(nr)
        else:
            skip_stats["csv_skipped"] += 1
    return out

def load_from_filtered_jsons(run_dir: str, skip_stats: Dict[str, int]) -> List[Dict[str, Any]]:
    base = os.path.join(run_dir, "exchange_rates_json_filtered")
    if not os.path.isdir(base):
        return []
    out: List[Dict[str, Any]] = []

    for name in os.listdir(base):
        if not name.lower().endswith(".json"):
            continue
        p = os.path.join(base, name)
        fallback_iso = ""
        m = FNAME_DATE_RE.match(name)
        if m:
            yyyy, mm, dd = m.group(1), m.group(2), m.group(3)
            fallback_iso = f"{yyyy}-{mm}-{dd}"

        try:
            with open(p, "r", encoding="utf-8") as f:
                obj = json.load(f)
        except Exception:
            skip_stats["bad_json"] += 1
            continue

        if isinstance(obj, dict) and isinstance(obj.get("data"), list):
            iterable = obj["data"]
        elif isinstance(obj, list):
            iterable = obj
        elif isinstance(obj, dict):
            iterable = [obj]
        else:
            skip_stats["unknown_shape"] += 1
            continue

        for item in iterable:
            if isinstance(item, dict):
                nr = normalize_row_from_json(item, fallback_iso_date=fallback_iso)
                if nr:
                    out.append(nr)
                else:
                    skip_stats["json_row_skipped"] += 1
            else:
                skip_stats["non_dict_row"] += 1
    return out

def dedupe_sort(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    seen: Dict[Tuple[str, str, str, str, str], Dict[str, Any]] = {}
    for r in rows:
        key = (r["FromCurrency"], r["ToCurrency"], r["ValidFrom"], r["ExchangeRateType"], r["Quotation"])
        seen[key] = r
    uniq = list(seen.values())

    def _key(rw: Dict[str, Any]) -> Tuple[str, str, str]:
        m = re.fullmatch(r"(\d{2})\.(\d{2})\.(\d{4})", rw["ValidFrom"])
        ymd = f"{m.group(3)}{m.group(2)}{m.group(1)}" if m else "00000000"
        return (ymd, rw["FromCurrency"], rw["ToCurrency"])

    uniq.sort(key=_key)
    return uniq

# --- NEW: duplicate each logical row into M and P variants (before rounding) ---
def _duplicate_rows_M_and_P(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    out: List[Dict[str, Any]] = []
    for r in rows:
        base = {
            "FromCurrency": r.get("FromCurrency"),
            "ToCurrency": r.get("ToCurrency"),
            "ValidFrom": r.get("ValidFrom"),
            "Quotation": r.get("Quotation"),
            "ExchangeRate": r.get("ExchangeRate"),
        }
        m_obj = dict(base)
        m_obj["ExchangeRateType"] = "M"
        p_obj = dict(base)
        p_obj["ExchangeRateType"] = "P"
        out.append(m_obj)
        out.append(p_obj)
    return out
# ------------------------------------------------------------------------------

def build_payloads_for_all_days() -> List[str]:
    written_paths: List[str] = []
    day_dirs = list_all_day_dirs(BASE_DIR)
    if not day_dirs:
        print(f"No day directories under {BASE_DIR} matching YYYY-MM-DD")
        return written_paths

    rounding_rules = load_rounding_rules(CURRENCY_RULES_CSV)
    if rounding_rules:
        print(f"[rounding] loaded {len(rounding_rules)} currency rules from {CURRENCY_RULES_CSV}")
    else:
        print(f"[rounding] no rules found (rates will not be rounded)")

    for run_dir in day_dirs:
        print(f"[run-dir] {run_dir}")
        skip_stats = {
            "csv_skipped": 0,
            "bad_json": 0,
            "unknown_shape": 0,
            "json_row_skipped": 0,
            "non_dict_row": 0,
        }

        rows = load_from_filtered_csv(run_dir, skip_stats)
        source = "CSV(filtered)"
        if not rows:
            rows = load_from_filtered_jsons(run_dir, skip_stats)
            source = "JSON(filtered)"

        # ✱ Fail fast if neither filtered source is present.
        if not rows:
            raise SystemExit(
                f"[FATAL] No filtered inputs found in {run_dir}.\n"
                f"Expected one of:\n"
                f"  - {os.path.join(run_dir, 'exchange_rates_agg.filtered.csv')}\n"
                f"  - {os.path.join(run_dir, 'exchange_rates_json_filtered')}/<*.json>\n"
                f"Run filter_to_needed_pairs.py AFTER fetch and BEFORE build."
            )

        # Duplicate every logical row into both M and P BEFORE rounding
        rows = _duplicate_rows_M_and_P(rows)

        # Apply rounding with zero-exception safeguard (covers both M and P)
        zero_exceptions = apply_rounding(rows, rounding_rules)

        # If any rounding would have produced zero, persist them to zero_exception.json
        if zero_exceptions:
            zp = os.path.join(run_dir, "zero_exception.json")
            try:
                with open(zp, "w", encoding="utf-8") as zf:
                    json.dump(zero_exceptions, zf, ensure_ascii=False, indent=2)
                print(f"[rounding] zero-exceptions saved: {len(zero_exceptions)} → {zp}")
            except Exception as ex:
                print(f"[warn] failed to write zero_exception.json in {run_dir}: {ex}")
        else:
            print(f"[rounding] zero-exceptions: 0")

        rows = dedupe_sort(rows)
        print(f"[source] {source}, rows={len(rows)}; skips={skip_stats}")

        out_path = os.path.join(run_dir, "exchange_rates_payload.json")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(rows, f, ensure_ascii=False, indent=2)
        print(f"[written] {out_path}")
        written_paths.append(out_path)

    print(f"[summary] payloads written: {len(written_paths)}")
    return written_paths

if __name__ == "__main__":
    build_payloads_for_all_days()


[rounding] loaded 31 currency rules from WebService/Currency.csv
[run-dir] WebService/data\2025-07-01
[rounding] zero-exceptions saved: 18 → WebService/data\2025-07-01\zero_exception.json
[source] CSV(filtered), rows=398; skips={'csv_skipped': 0, 'bad_json': 0, 'unknown_shape': 0, 'json_row_skipped': 0, 'non_dict_row': 0}
[written] WebService/data\2025-07-01\exchange_rates_payload.json
[run-dir] WebService/data\2025-07-02
[rounding] zero-exceptions saved: 18 → WebService/data\2025-07-02\zero_exception.json
[source] CSV(filtered), rows=398; skips={'csv_skipped': 0, 'bad_json': 0, 'unknown_shape': 0, 'json_row_skipped': 0, 'non_dict_row': 0}
[written] WebService/data\2025-07-02\exchange_rates_payload.json
[run-dir] WebService/data\2025-07-03
[rounding] zero-exceptions saved: 18 → WebService/data\2025-07-03\zero_exception.json
[source] CSV(filtered), rows=398; skips={'csv_skipped': 0, 'bad_json': 0, 'unknown_shape': 0, 'json_row_skipped': 0, 'non_dict_row': 0}
[written] WebService/data\20

## Temp Build Code

In [1]:
import os
import re
import csv
import json
from decimal import Decimal, ROUND_HALF_UP, InvalidOperation
from typing import Optional, List, Dict, Any, Tuple

BASE_DIR = "WebService/data"
CURRENCY_RULES_CSV = "WebService/Currency.csv"  # round_digits,target_currency
RUN_DIR_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}$")
FNAME_DATE_RE = re.compile(r"^(\d{4})-(\d{2})-(\d{2})\.json$", re.IGNORECASE)

def list_all_day_dirs(base_dir: str) -> List[str]:
    if not os.path.isdir(base_dir):
        return []
    out: List[str] = []
    for name in os.listdir(base_dir):
        p = os.path.join(base_dir, name)
        if os.path.isdir(p) and RUN_DIR_PATTERN.match(name):
            out.append(p)
    out.sort()
    return out

def _norm_ccy(x: Any) -> str:
    s = (x or "").strip().upper()
    return s if re.fullmatch(r"[A-Z]{3,4}", s) else ""

def _parse_rate(x: Any) -> Optional[float]:
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    sx = str(x).strip().replace(",", "")
    try:
        return float(sx)
    except ValueError:
        return None

def _is_mmddyyyy(s: str) -> bool:
    return bool(re.fullmatch(r"\d{2}/\d{2}/\d{4}", s or ""))

def _to_ddmmyyyy_from_iso(iso: str) -> str:
    m = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", iso or "")
    if not m:
        return ""
    yyyy, mm, dd = m.group(1), m.group(2), m.group(3)
    return f"{dd}.{mm}.{yyyy}"

def _mmddyyyy_to_ddmmyyyy_dots(s: str) -> str:
    m = re.fullmatch(r"(\d{2})/(\d{2})/(\d{4})", s or "")
    if not m:
        return ""
    mm, dd, yyyy = m.group(1), m.group(2), m.group(3)
    return f"{dd}.{mm}.{yyyy}"

def _key_ci(d: Dict[str, Any], *cands: str) -> Optional[str]:
    lm = {k.lower(): k for k in d.keys()}
    for c in cands:
        k = lm.get(c.lower())
        if k is not None:
            return k
    return None

def load_rounding_rules(csv_path: str) -> Dict[str, int]:
    rules: Dict[str, int] = {}
    if not os.path.isfile(csv_path):
        print(f"[warn] rounding rules CSV not found: {csv_path} (rates will not be rounded)")
        return rules
    with open(csv_path, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        for row in rdr:
            if not isinstance(row, dict):
                continue
            lower_map = {k.lower(): k for k in row.keys()}
            k_digits = lower_map.get("round_digits")
            k_ccy    = lower_map.get("target_currency")
            if not k_digits or not k_ccy:
                continue
            try:
                digits = int(str(row[k_digits]).strip())
            except (ValueError, TypeError):
                continue
            ccy = _norm_ccy(row[k_ccy])
            if not ccy:
                continue
            if digits < 0: digits = 0
            if digits > 12: digits = 12
            rules[ccy] = digits
    return rules

def _round_rate_half_up(rate: float, digits: int) -> float:
    try:
        q = Decimal("1").scaleb(-digits)
        d = Decimal(str(rate)).quantize(q, rounding=ROUND_HALF_UP)
        return float(d)
    except (InvalidOperation, ValueError):
        return rate

def apply_rounding(rows: List[Dict[str, Any]], rules: Dict[str, int]) -> List[Dict[str, Any]]:
    """
    Apply rounding per rules to rows *in-place*.
    NEW: If rounding would turn a non-zero rate into 0.0, keep the original rate
    and record the case in a zero-exception list (returned).
    Returns:
        zero_exceptions: list of dicts describing rows where rounding would yield zero.
    """
    zero_exceptions: List[Dict[str, Any]] = []
    if not rules:
        return zero_exceptions

    for r in rows:
        try:
            to_ccy = _norm_ccy(r.get("ToCurrency"))
            if not to_ccy:
                continue
            digits = rules.get(to_ccy)
            if digits is None:
                continue
            rate = r.get("ExchangeRate")
            if rate is None:
                continue
            rate_f = _parse_rate(rate)
            if rate_f is None:
                continue

            rounded = _round_rate_half_up(rate_f, digits)

            # If rounding makes a non-zero rate become zero, DON'T round.
            if rate_f != 0.0 and rounded == 0.0:
                zero_exceptions.append({
                    "FromCurrency": r.get("FromCurrency"),
                    "ToCurrency": r.get("ToCurrency"),
                    "ValidFrom": r.get("ValidFrom"),
                    "ExchangeRateType": r.get("ExchangeRateType"),
                    "Quotation": r.get("Quotation"),
                    "original_exchange_rate": rate_f,
                    "attempted_digits": digits,
                    "would_round_to": rounded
                })
                # keep original: do NOT assign rounded
                continue

            # Normal case: apply rounded value
            r["ExchangeRate"] = rounded

        except Exception:
            # Swallow per-row errors to avoid breaking the whole day; no logging change here.
            continue

    return zero_exceptions

def normalize_row_from_csv(raw: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    k_from = _key_ci(raw, "FromCurrency", "sourceCurrency", "From", "BaseCurrency", "SourceCurrency")
    k_to   = _key_ci(raw, "ToCurrency", "targetCurrency", "To", "QuoteCurrency", "TargetCurrency")
    fcur = _norm_ccy(raw.get(k_from, "")) if k_from else ""
    tcur = _norm_ccy(raw.get(k_to, ""))   if k_to else ""
    if not fcur or not tcur:
        return None

    k_rate = _key_ci(raw, "exChangeRateValue", "ExchangeRate", "Rate", "FxRate", "Value")
    rate = _parse_rate(raw.get(k_rate)) if k_rate else None
    if rate is None:
        return None

    k_date_csv = _key_ci(raw, "exChangeRateDate")
    valid_from = ""
    if k_date_csv:
        cand = (raw.get(k_date_csv) or "").strip()
        if _is_mmddyyyy(cand):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(cand)

    if not valid_from:
        k_iso = _key_ci(raw, "_service_date")
        if k_iso and raw.get(k_iso):
            valid_from = _to_ddmmyyyy_from_iso(str(raw[k_iso]).strip())

    if not valid_from:
        return None

    return {
        "ExchangeRateType": "M",
        "FromCurrency": fcur,
        "ToCurrency": tcur,
        "ValidFrom": valid_from,
        "Quotation": "Direct",
        "ExchangeRate": rate,
    }

def normalize_row_from_json(raw: Dict[str, Any], fallback_iso_date: str = "") -> Optional[Dict[str, Any]]:
    k_from = _key_ci(raw, "FromCurrency", "sourceCurrency", "From", "BaseCurrency", "SourceCurrency")
    k_to   = _key_ci(raw, "ToCurrency", "targetCurrency", "To", "QuoteCurrency", "TargetCurrency")
    fcur = _norm_ccy(raw.get(k_from, "")) if k_from else ""
    tcur = _norm_ccy(raw.get(k_to, ""))   if k_to else ""
    if not fcur or not tcur:
        return None

    k_rate = _key_ci(raw, "exChangeRateValue", "ExchangeRate", "Rate", "FxRate", "Value")
    rate = _parse_rate(raw.get(k_rate)) if k_rate else None
    if rate is None:
        return None

    k_mmdd = _key_ci(raw, "exChangeRateDate")
    k_other= _key_ci(raw, "ValidFrom", "Date", "RateDate", "_service_date")
    valid_from = ""
    if k_mmdd:
        cand = (raw.get(k_mmdd) or "").strip()
        if _is_mmddyyyy(cand):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(cand)

    if not valid_from and k_other and raw.get(k_other):
        s = str(raw[k_other]).strip()
        if _is_mmddyyyy(s):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(s)
        else:
            valid_from = _to_ddmmyyyy_from_iso(s)

    if not valid_from and fallback_iso_date:
        valid_from = _to_ddmmyyyy_from_iso(fallback_iso_date)

    if not valid_from:
        return None

    k_type = _key_ci(raw, "ExchangeRateType", "Type")
    k_quo  = _key_ci(raw, "Quotation", "QuoteType", "Side")
    etype = str(raw.get(k_type)).strip() if (k_type and raw.get(k_type)) else "M"
    quo   = str(raw.get(k_quo)).strip()  if (k_quo  and raw.get(k_quo))  else "Direct"

    return {
        "ExchangeRateType": etype,
        "FromCurrency": fcur,
        "ToCurrency": tcur,
        "ValidFrom": valid_from,
        "Quotation": quo,
        "ExchangeRate": rate,
    }

def load_from_filtered_csv(run_dir: str, skip_stats: Dict[str, int]) -> List[Dict[str, Any]]:
    path = os.path.join(run_dir, "exchange_rates_agg.filtered.csv")
    if not os.path.isfile(path):
        return []
    with open(path, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        rows_raw = list(rdr)

    out: List[Dict[str, Any]] = []
    for r in rows_raw:
        nr = normalize_row_from_csv(r)
        if nr:
            out.append(nr)
        else:
            skip_stats["csv_skipped"] += 1
    return out

def load_from_filtered_jsons(run_dir: str, skip_stats: Dict[str, int]) -> List[Dict[str, Any]]:
    base = os.path.join(run_dir, "exchange_rates_json_filtered")
    if not os.path.isdir(base):
        return []
    out: List[Dict[str, Any]] = []

    for name in os.listdir(base):
        if not name.lower().endswith(".json"):
            continue
        p = os.path.join(base, name)
        fallback_iso = ""
        m = FNAME_DATE_RE.match(name)
        if m:
            yyyy, mm, dd = m.group(1), m.group(2), m.group(3)
            fallback_iso = f"{yyyy}-{mm}-{dd}"

        try:
            with open(p, "r", encoding="utf-8") as f:
                obj = json.load(f)
        except Exception:
            skip_stats["bad_json"] += 1
            continue

        if isinstance(obj, dict) and isinstance(obj.get("data"), list):
            iterable = obj["data"]
        elif isinstance(obj, list):
            iterable = obj
        elif isinstance(obj, dict):
            iterable = [obj]
        else:
            skip_stats["unknown_shape"] += 1
            continue

        for item in iterable:
            if isinstance(item, dict):
                nr = normalize_row_from_json(item, fallback_iso_date=fallback_iso)
                if nr:
                    out.append(nr)
                else:
                    skip_stats["json_row_skipped"] += 1
            else:
                skip_stats["non_dict_row"] += 1
    return out

def dedupe_sort(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    seen: Dict[Tuple[str, str, str, str, str], Dict[str, Any]] = {}
    for r in rows:
        key = (r["FromCurrency"], r["ToCurrency"], r["ValidFrom"], r["ExchangeRateType"], r["Quotation"])
        seen[key] = r
    uniq = list(seen.values())

    def _key(rw: Dict[str, Any]) -> Tuple[str, str, str]:
        m = re.fullmatch(r"(\d{2})\.(\d{2})\.(\d{4})", rw["ValidFrom"])
        ymd = f"{m.group(3)}{m.group(2)}{m.group(1)}" if m else "00000000"
        return (ymd, rw["FromCurrency"], rw["ToCurrency"])

    uniq.sort(key=_key)
    return uniq

# --- NEW: duplicate each logical row into M and P variants (before rounding) ---
def _duplicate_rows_M_and_P(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    out: List[Dict[str, Any]] = []
    for r in rows:
        base = {
            "FromCurrency": r.get("FromCurrency"),
            "ToCurrency": r.get("ToCurrency"),
            "ValidFrom": r.get("ValidFrom"),
            "Quotation": r.get("Quotation"),
            "ExchangeRate": r.get("ExchangeRate"),
        }
        m_obj = dict(base)
        m_obj["ExchangeRateType"] = "M"
        p_obj = dict(base)
        p_obj["ExchangeRateType"] = "P"
        out.append(m_obj)
        out.append(p_obj)
    return out
# ------------------------------------------------------------------------------

# --- NEW: global exclusions (applied across ALL days) --------------------------
# Drop these PAIRS for ExchangeRateType == "P" regardless of date/rate.
EXCLUDED_P_PAIRS: set[Tuple[str, str, str]] = {
    ("P", "SAR", "IQD"),
    ("P", "JOD", "CHF"),
    ("P", "JOD", "GBP"),
    ("P", "JOD", "JPY"),
    ("P", "IQD", "AED"),
    ("P", "IQD", "EUR"),
    ("P", "IQD", "JOD"),
    ("P", "IQD", "SAR"),
    ("P", "GBP", "JOD"),
    ("P", "AED", "JOD"),
    ("P", "AED", "IQD"),
    ("P", "IQD", "USD"),
}

def _filter_excluded_pairs(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    keep: List[Dict[str, Any]] = []
    dropped = 0
    for r in rows:
        et = str(r.get("ExchangeRateType", "")).strip().upper()
        fc = _norm_ccy(r.get("FromCurrency"))
        tc = _norm_ccy(r.get("ToCurrency"))
        if (et, fc, tc) in EXCLUDED_P_PAIRS:
            dropped += 1
            continue
        keep.append(r)
    if dropped:
        print(f"[exclude] dropped {dropped} P-pair rows by rule")
    else:
        print(f"[exclude] dropped 0 rows")
    return keep
# ------------------------------------------------------------------------------

def build_payloads_for_all_days() -> List[str]:
    written_paths: List[str] = []
    day_dirs = list_all_day_dirs(BASE_DIR)
    if not day_dirs:
        print(f"No day directories under {BASE_DIR} matching YYYY-MM-DD")
        return written_paths

    rounding_rules = load_rounding_rules(CURRENCY_RULES_CSV)
    if rounding_rules:
        print(f"[rounding] loaded {len(rounding_rules)} currency rules from {CURRENCY_RULES_CSV}")
    else:
        print(f"[rounding] no rules found (rates will not be rounded)")

    for run_dir in day_dirs:
        print(f"[run-dir] {run_dir}")
        skip_stats = {
            "csv_skipped": 0,
            "bad_json": 0,
            "unknown_shape": 0,
            "json_row_skipped": 0,
            "non_dict_row": 0,
        }

        rows = load_from_filtered_csv(run_dir, skip_stats)
        source = "CSV(filtered)"
        if not rows:
            rows = load_from_filtered_jsons(run_dir, skip_stats)
            source = "JSON(filtered)"

        # ✱ Fail fast if neither filtered source is present.
        if not rows:
            raise SystemExit(
                f"[FATAL] No filtered inputs found in {run_dir}.\n"
                f"Expected one of:\n"
                f"  - {os.path.join(run_dir, 'exchange_rates_agg.filtered.csv')}\n"
                f"  - {os.path.join(run_dir, 'exchange_rates_json_filtered')}/<*.json>\n"
                f"Run filter_to_needed_pairs.py AFTER fetch and BEFORE build."
            )

        # Duplicate every logical row into both M and P BEFORE rounding
        rows = _duplicate_rows_M_and_P(rows)

        # NEW: drop the specified P pairs for ALL days before rounding/zero-exception logic
        rows = _filter_excluded_pairs(rows)

        # Apply rounding with zero-exception safeguard (covers both M and P)
        zero_exceptions = apply_rounding(rows, rounding_rules)

        # If any rounding would have produced zero, persist them to zero_exception.json
        if zero_exceptions:
            zp = os.path.join(run_dir, "zero_exception.json")
            try:
                with open(zp, "w", encoding="utf-8") as zf:
                    json.dump(zero_exceptions, zf, ensure_ascii=False, indent=2)
                print(f"[rounding] zero-exceptions saved: {len(zero_exceptions)} → {zp}")
            except Exception as ex:
                print(f"[warn] failed to write zero_exception.json in {run_dir}: {ex}")
        else:
            print(f"[rounding] zero-exceptions: 0")

        rows = dedupe_sort(rows)
        print(f"[source] {source}, rows={len(rows)}; skips={skip_stats}")

        out_path = os.path.join(run_dir, "exchange_rates_payload.json")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(rows, f, ensure_ascii=False, indent=2)
        print(f"[written] {out_path}")
        written_paths.append(out_path)

    print(f"[summary] payloads written: {len(written_paths)}")
    return written_paths

if __name__ == "__main__":
    build_payloads_for_all_days()


[rounding] loaded 31 currency rules from WebService/Currency.csv
[run-dir] WebService/data\2025-07-01
[exclude] dropped 12 P-pair rows by rule
[rounding] zero-exceptions saved: 14 → WebService/data\2025-07-01\zero_exception.json
[source] CSV(filtered), rows=386; skips={'csv_skipped': 0, 'bad_json': 0, 'unknown_shape': 0, 'json_row_skipped': 0, 'non_dict_row': 0}
[written] WebService/data\2025-07-01\exchange_rates_payload.json
[run-dir] WebService/data\2025-07-02
[exclude] dropped 12 P-pair rows by rule
[rounding] zero-exceptions saved: 14 → WebService/data\2025-07-02\zero_exception.json
[source] CSV(filtered), rows=386; skips={'csv_skipped': 0, 'bad_json': 0, 'unknown_shape': 0, 'json_row_skipped': 0, 'non_dict_row': 0}
[written] WebService/data\2025-07-02\exchange_rates_payload.json
[run-dir] WebService/data\2025-07-03
[exclude] dropped 12 P-pair rows by rule
[rounding] zero-exceptions saved: 14 → WebService/data\2025-07-03\zero_exception.json
[source] CSV(filtered), rows=386; skips={

# Fixer

In [ ]:
# fetch_fixer_range.py
# pip install requests pandas
import os
import re
import json
import csv
import time
from datetime import date, timedelta, datetime
from typing import Iterable, Dict, Any, List, Optional, Tuple, Set

import requests
from requests.exceptions import ConnectTimeout, ReadTimeout, ProxyError, SSLError, ConnectionError as ReqConnectionError

# === CONFIG ===
ACCESS_KEY = "4e24104d947d9a92ecad3d7c44059f9f"
FIXER_BASE_URL = "https://data.fixer.io/api"  # will call /YYYY-MM-DD
DATE_START = date(2025, 7, 16)
DATE_END   = date(2025, 7, 31)

NEEDED_FILE = os.path.join("WebService", "needed_rates.txt")  # lines like: EUR/JOD
BASE_SAVE_DIR = "WebService"

# Timeouts / retries (HTTP)
CONNECT_TIMEOUT = 5
READ_TIMEOUT = 30
MAX_RETRIES = 3
RETRY_BACKOFF_BASE = 2

DATE_RE_MMDDYYYY = re.compile(r"^\d{2}/\d{2}/\d{4}$")

# === Utilities ===
def mmddyyyy(d: date) -> str:
    return f"{d.month:02d}/{d.day:02d}/{d.year:04d}"

def daterange_days(start: date, end: date) -> Iterable[date]:
    d = start
    while d <= end:
        yield d
        d += timedelta(days=1)

def ensure_dir(p: str) -> None:
    if not os.path.isdir(p):
        os.makedirs(p, exist_ok=True)

def _make_session() -> requests.Session:
    s = requests.Session()
    s.headers.update({"Accept": "application/json"})
    s.trust_env = False
    s.proxies.update({"http": None, "https": None})
    return s

def _fetch_fixer_day(d: date, symbols: List[str], session: requests.Session) -> Dict[str, Any]:
    url = f"{FIXER_BASE_URL}/{d.isoformat()}"
    params = {"access_key": ACCESS_KEY, "symbols": ",".join(sorted(set(symbols)))}
    last_err: Optional[Exception] = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = session.get(url, params=params, timeout=(CONNECT_TIMEOUT, READ_TIMEOUT))
            r.raise_for_status()
            data = r.json()
            if not data.get("success", False):
                raise RuntimeError(f"Fixer error: {data.get('error', data)}")
            return data
        except (ConnectTimeout, ReadTimeout, ProxyError, SSLError, ReqConnectionError, requests.HTTPError) as e:
            last_err = e
            time.sleep(min(RETRY_BACKOFF_BASE ** attempt, 8))
        except Exception as e:
            last_err = e
            time.sleep(min(RETRY_BACKOFF_BASE ** attempt, 8))
    raise RuntimeError(f"[{d}] Failed after {MAX_RETRIES} attempts: {last_err}")

def _read_needed_pairs(path: str) -> List[Tuple[str, str]]:
    pairs: List[Tuple[str, str]] = []
    if not os.path.isfile(path):
        raise FileNotFoundError(f"needed_rates file not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s or s.startswith("#"):
                continue
            if "/" not in s:
                parts = s.replace(" ", "/").split("/")
            else:
                parts = s.split("/")
            if len(parts) != 2:
                continue
            a = parts[0].strip().upper()
            b = parts[1].strip().upper()
            if re.fullmatch(r"[A-Z]{3,4}", a) and re.fullmatch(r"[A-Z]{3,4}", b):
                pairs.append((a, b))
    if not pairs:
        raise ValueError("No valid pairs loaded from needed_rates.txt")
    return pairs

def _symbols_from_pairs(pairs: List[Tuple[str, str]]) -> List[str]:
    s: Set[str] = set()
    for a, b in pairs:
        s.add(a); s.add(b)
    # Free Fixer base is EUR; include EUR to be safe in math
    s.add("EUR")
    return sorted(s)

def _compute_pair_rate(a: str, b: str, rates_vs_eur: Dict[str, float]) -> Optional[float]:
    # A->B = (EUR->B)/(EUR->A)
    ra = rates_vs_eur.get(a)
    rb = rates_vs_eur.get(b)
    if ra is None or rb is None or ra == 0:
        return None
    return rb / ra

def _save_day_json(out_dir: str, d: date, rows: List[Dict[str, Any]]) -> str:
    json_dir = os.path.join(out_dir, "exchange_rates_json")
    ensure_dir(json_dir)
    p = os.path.join(json_dir, f"{d.isoformat()}.json")
    with open(p, "w", encoding="utf-8") as f:
        json.dump({"data": rows}, f, ensure_ascii=False, indent=2)
    return p

def _append_csv(out_dir: str, rows: List[Dict[str, Any]]) -> None:
    csv_path = os.path.join(out_dir, "exchange_rates_agg.csv")
    if not rows:
        return
    # discover headers
    all_keys = set()
    if os.path.exists(csv_path):
        with open(csv_path, "r", newline="", encoding="utf-8") as f:
            rdr = csv.DictReader(f)
            all_keys.update(rdr.fieldnames or [])
    for r in rows:
        all_keys.update(r.keys())
    fieldnames = sorted(all_keys)

    # load existing
    existing: List[Dict[str, Any]] = []
    if os.path.exists(csv_path):
        with open(csv_path, "r", newline="", encoding="utf-8") as f:
            rdr = csv.DictReader(f)
            for r in rdr:
                existing.append(r)

    ensure_dir(out_dir)
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in existing:
            w.writerow({k: r.get(k, "") for k in fieldnames})
        for r in rows:
            w.writerow({k: r.get(k, "") for k in fieldnames})

def main():
    pairs = _read_needed_pairs(NEEDED_FILE)
    symbols = _symbols_from_pairs(pairs)
    session = _make_session()

    for d in daterange_days(DATE_START, DATE_END):
        day_dir = os.path.join(BASE_SAVE_DIR, d.isoformat())
        ensure_dir(day_dir)

        # fetch once per day with all needed symbols
        data = _fetch_fixer_day(d, symbols, session)
        rates = dict(data.get("rates", {}))
        rates["EUR"] = 1.0  # guarantee

        # compute only requested pairs; output unified row shape your later steps expect
        mmdd = mmddyyyy(d)
        day_rows: List[Dict[str, Any]] = []
        skipped: List[Tuple[str, str]] = []

        for a, b in pairs:
            r = _compute_pair_rate(a, b, rates)
            if r is None:
                skipped.append((a, b))
                continue
            row = {
                "ExchangeRateType": "M",
                "FromCurrency": a,
                "ToCurrency": b,
                "exChangeRateDate": mmdd,      # MM/DD/YYYY
                "Quotation": "Direct",
                "exChangeRateValue": round(float(r), 9),
                "_source": "Fixer",
                "_fixer_date": data.get("date", d.isoformat()),
                "_base": "EUR"
            }
            day_rows.append(row)

        # save JSON + CSV inside the day folder
        _save_day_json(day_dir, d, day_rows)
        _append_csv(day_dir, day_rows)

        # simple log
        print(f"[{d}] kept={len(day_rows)} skipped={len(skipped)} → {day_dir}")

if __name__ == "__main__":
    main()


In [ ]:
# filter_to_needed_pairs.py
import os
import re
import json
import csv
from typing import Dict, Any, List, Tuple, Set

BASE_DIR = "WebService"
NEEDED_FILE = os.path.join("Weneeded_rates.txt")  # AAA/BBB per line

# match single-day directories: YYYY-MM-DD
RUN_DIR_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}$")

def ensure_dir(p: str) -> None:
    if not os.path.isdir(p):
        os.makedirs(p, exist_ok=True)

def _norm_currency(s: str) -> str:
    return (s or "").strip().upper()

def _find_key_case_insensitive(d: Dict[str, Any], *candidates: str) -> str:
    lower_map = {k.lower(): k for k in d.keys()}
    for c in candidates:
        k = lower_map.get(c.lower())
        if k is not None:
            return k
    return ""

def _row_pair(row: Dict[str, Any]) -> Tuple[str, str]:
    if not isinstance(row, dict):
        return "", ""
    from_k = _find_key_case_insensitive(row, "FromCurrency", "From", "BaseCurrency", "SourceCurrency")
    to_k   = _find_key_case_insensitive(row, "ToCurrency", "To", "QuoteCurrency", "TargetCurrency")
    f = _norm_currency(row.get(from_k, "")) if from_k else ""
    t = _norm_currency(row.get(to_k, ""))   if to_k else ""
    return f, t

def load_needed_pairs(path: str) -> Set[Tuple[str, str]]:
    pairs: Set[Tuple[str, str]] = set()
    if not os.path.isfile(path):
        print(f"[warn] needed list not found: {path} (no filtering will be applied)")
        return pairs
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s or s.startswith("#"):
                continue
            if "/" not in s:
                parts = s.replace(" ", "/").split("/")
            else:
                parts = s.split("/")
            if len(parts) != 2:
                continue
            a, b = _norm_currency(parts[0]), _norm_currency(parts[1])
            if a and b:
                pairs.add((a, b))
    return pairs

def should_keep(row: Dict[str, Any], include_set: Set[Tuple[str, str]]) -> bool:
    if not include_set:
        return True  # if list missing, keep everything
    f, t = _row_pair(row)
    if not f or not t:
        return False
    return (f, t) in include_set

def filter_json_object(obj: Any, include_set: Set[Tuple[str, str]]) -> Any:
    try:
        if isinstance(obj, list):
            return [r for r in obj if not isinstance(r, dict) or should_keep(r, include_set)]
        if isinstance(obj, dict):
            if "data" in obj and isinstance(obj["data"], list):
                new_obj = dict(obj)
                new_obj["data"] = [r for r in obj["data"] if not isinstance(r, dict) or should_keep(r, include_set)]
                return new_obj
            # single object → keep only if it matches; else empty list-like stub
            return obj if should_keep(obj, include_set) else {"data": []}
        return obj
    except Exception:
        return obj

def filter_all_jsons(run_dir: str, include_set: Set[Tuple[str, str]]) -> Tuple[int, int]:
    json_in = os.path.join(run_dir, "exchange_rates_json")
    json_out = os.path.join(run_dir, "exchange_rates_json_filtered")
    ensure_dir(json_out)

    processed = 0
    written = 0

    if not os.path.isdir(json_in):
        print(f"[warn] JSON input dir not found: {json_in}")
        return (0, 0)

    for name in os.listdir(json_in):
        if not name.lower().endswith(".json"):
            continue
        inp = os.path.join(json_in, name)
        out = os.path.join(json_out, name)
        try:
            with open(inp, "r", encoding="utf-8") as f:
                obj = json.load(f)
            new_obj = filter_json_object(obj, include_set)
            with open(out, "w", encoding="utf-8") as f:
                json.dump(new_obj, f, ensure_ascii=False, indent=2)
            processed += 1
            written += 1
        except Exception as e:
            print(f"[json] skip {inp}: {e}")
            processed += 1
    return processed, written

def filter_csv(run_dir: str, include_set: Set[Tuple[str, str]]) -> Tuple[int, int]:
    csv_in = os.path.join(run_dir, "exchange_rates_agg.csv")
    csv_out = os.path.join(run_dir, "exchange_rates_agg.filtered.csv")

    if not os.path.isfile(csv_in):
        print(f"[warn] CSV input not found: {csv_in}")
        return (0, 0)

    with open(csv_in, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        fieldnames = rdr.fieldnames or []
        rows = list(rdr)

    if not fieldnames:
        print(f"[warn] CSV has no header: {csv_in}")
        return (0, 0)

    kept: List[Dict[str, Any]] = []
    for r in rows:
        if should_keep(r, include_set):
            kept.append(r)

    # Ensure FromCurrency/ToCurrency exist in header
    if "FromCurrency" not in fieldnames:
        fieldnames = fieldnames + ["FromCurrency"]
    if "ToCurrency" not in fieldnames:
        fieldnames = fieldnames + ["ToCurrency"]

    with open(csv_out, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in kept:
            out_row = {k: r.get(k, "") for k in fieldnames}
            fcur, tcur = _row_pair(r)
            if fcur and not out_row.get("FromCurrency"):
                out_row["FromCurrency"] = fcur
            if tcur and not out_row.get("ToCurrency"):
                out_row["ToCurrency"] = tcur
            w.writerow(out_row)

    return (len(rows), len(kept))

def list_all_day_dirs(base_dir: str) -> List[str]:
    if not os.path.isdir(base_dir):
        return []
    out: List[str] = []
    for name in os.listdir(base_dir):
        p = os.path.join(base_dir, name)
        if os.path.isdir(p) and RUN_DIR_PATTERN.match(name):
            out.append(p)
    out.sort()  # chronological by name
    return out

# === RUN ===
if __name__ == "__main__":
    day_dirs = list_all_day_dirs(BASE_DIR)
    if not day_dirs:
        raise SystemExit(f"No day directories found under {BASE_DIR} matching YYYY-MM-DD")

    needed = load_needed_pairs(NEEDED_FILE)
    total_json_in = total_json_out = total_csv_in = total_csv_out = 0

    for run_dir in day_dirs:
        print(f"\n[run-dir] {run_dir}")
        j_in, j_out = filter_all_jsons(run_dir, needed)
        print(f"[done] JSON files processed={j_in}, written={j_out} → {os.path.join(run_dir, 'exchange_rates_json_filtered')}")
        c_in, c_out = filter_csv(run_dir, needed)
        print(f"[done] CSV rows in={c_in}, out={c_out} → {os.path.join(run_dir, 'exchange_rates_agg.filtered.csv')}")
        total_json_in += j_in; total_json_out += j_out
        total_csv_in  += c_in; total_csv_out  += c_out

    print(f"\n[summary] JSON in={total_json_in}, JSON out={total_json_out}; CSV in={total_csv_in}, CSV out={total_csv_out}")


In [ ]:
# build_payload.py
import os
import re
import csv
import json
from typing import Optional, List, Dict, Any, Tuple

BASE_DIR = "WebService"
RUN_DIR_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}$")
FNAME_DATE_RE = re.compile(r"^(\d{4})-(\d{2})-(\d{2})\.json$", re.IGNORECASE)

# ---------- utils ----------
def list_all_day_dirs(base_dir: str) -> List[str]:
    if not os.path.isdir(base_dir):
        return []
    out: List[str] = []
    for name in os.listdir(base_dir):
        p = os.path.join(base_dir, name)
        if os.path.isdir(p) and RUN_DIR_PATTERN.match(name):
            out.append(p)
    out.sort()  # chronological
    return out

def _norm_ccy(x: Any) -> str:
    s = (x or "").strip().upper()
    return s if re.fullmatch(r"[A-Z]{3,4}", s) else ""

def _parse_rate(x: Any) -> Optional[float]:
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    sx = str(x).strip().replace(",", "")
    try:
        return float(sx)
    except ValueError:
        return None

def _is_mmddyyyy(s: str) -> bool:
    return bool(re.fullmatch(r"\d{2}/\d{2}/\d{4}", s or ""))

def _to_ddmmyyyy_from_iso(iso: str) -> str:
    # "YYYY-MM-DD" -> "DD.MM.YYYY"
    m = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", iso or "")
    if not m:
        return ""
    yyyy, mm, dd = m.group(1), m.group(2), m.group(3)
    return f"{dd}.{mm}.{yyyy}"

def _mmddyyyy_to_ddmmyyyy_dots(s: str) -> str:
    # "MM/DD/YYYY" -> "DD.MM.YYYY"
    m = re.fullmatch(r"(\d{2})/(\d{2})/(\d{4})", s or "")
    if not m:
        return ""
    mm, dd, yyyy = m.group(1), m.group(2), m.group(3)
    return f"{dd}.{mm}.{yyyy}"

def _key_ci(d: Dict[str, Any], *cands: str) -> Optional[str]:
    lm = {k.lower(): k for k in d.keys()}
    for c in cands:
        k = lm.get(c.lower())
        if k is not None:
            return k
    return None

# ---------- row normalization ----------
def normalize_row_from_csv(raw: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """
    Map to:
    {
      "ExchangeRateType": "M",
      "FromCurrency": "...",
      "ToCurrency": "...",
      "ValidFrom": "DD.MM.YYYY",
      "Quotation": "Direct",
      "ExchangeRate": float
    }
    """
    k_from = _key_ci(raw, "FromCurrency", "sourceCurrency", "From", "BaseCurrency", "SourceCurrency")
    k_to   = _key_ci(raw, "ToCurrency", "targetCurrency", "To", "QuoteCurrency", "TargetCurrency")
    fcur = _norm_ccy(raw.get(k_from, "")) if k_from else ""
    tcur = _norm_ccy(raw.get(k_to, ""))   if k_to else ""
    if not fcur or not tcur:
        return None

    k_rate = _key_ci(raw, "exChangeRateValue", "ExchangeRate", "Rate", "FxRate", "Value")
    rate = _parse_rate(raw.get(k_rate)) if k_rate else None
    if rate is None:
        return None

    # Prefer exChangeRateDate if present (MM/DD/YYYY), else _service_date (YYYY-MM-DD)
    k_date_csv = _key_ci(raw, "exChangeRateDate")
    valid_from = ""
    if k_date_csv:
        cand = (raw.get(k_date_csv) or "").strip()
        if _is_mmddyyyy(cand):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(cand)

    if not valid_from:
        k_iso = _key_ci(raw, "_service_date")  # YYYY-MM-DD
        if k_iso and raw.get(k_iso):
            valid_from = _to_ddmmyyyy_from_iso(str(raw[k_iso]).strip())

    if not valid_from:
        return None

    return {
        "ExchangeRateType": "M",
        "FromCurrency": fcur,
        "ToCurrency": tcur,
        "ValidFrom": valid_from,  # DD.MM.YYYY
        "Quotation": "Direct",
        "ExchangeRate": rate,
    }

def normalize_row_from_json(raw: Dict[str, Any], fallback_iso_date: str = "") -> Optional[Dict[str, Any]]:
    k_from = _key_ci(raw, "FromCurrency", "sourceCurrency", "From", "BaseCurrency", "SourceCurrency")
    k_to   = _key_ci(raw, "ToCurrency", "targetCurrency", "To", "QuoteCurrency", "TargetCurrency")
    fcur = _norm_ccy(raw.get(k_from, "")) if k_from else ""
    tcur = _norm_ccy(raw.get(k_to, ""))   if k_to else ""
    if not fcur or not tcur:
        return None

    k_rate = _key_ci(raw, "exChangeRateValue", "ExchangeRate", "Rate", "FxRate", "Value")
    rate = _parse_rate(raw.get(k_rate)) if k_rate else None
    if rate is None:
        return None

    # Try MM/DD/YYYY field first
    k_mmdd = _key_ci(raw, "exChangeRateDate")
    k_other= _key_ci(raw, "ValidFrom", "Date", "RateDate", "_service_date")
    valid_from = ""
    if k_mmdd:
        cand = (raw.get(k_mmdd) or "").strip()
        if _is_mmddyyyy(cand):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(cand)

    # Then try other date fields (may be MM/DD/YYYY or ISO)
    if not valid_from and k_other and raw.get(k_other):
        s = str(raw[k_other]).strip()
        if _is_mmddyyyy(s):
            valid_from = _mmddyyyy_to_ddmmyyyy_dots(s)
        else:
            valid_from = _to_ddmmyyyy_from_iso(s)

    # Fallback to filename ISO date
    if not valid_from and fallback_iso_date:
        valid_from = _to_ddmmyyyy_from_iso(fallback_iso_date)

    if not valid_from:
        return None

    k_type = _key_ci(raw, "ExchangeRateType", "Type")
    k_quo  = _key_ci(raw, "Quotation", "QuoteType", "Side")
    etype = str(raw.get(k_type)).strip() if (k_type and raw.get(k_type)) else "M"
    quo   = str(raw.get(k_quo)).strip()  if (k_quo  and raw.get(k_quo))  else "Direct"

    return {
        "ExchangeRateType": etype,
        "FromCurrency": fcur,
        "ToCurrency": tcur,
        "ValidFrom": valid_from,  # DD.MM.YYYY
        "Quotation": quo,
        "ExchangeRate": rate,
    }

# ---------- loaders ----------
def load_from_filtered_csv(run_dir: str, skip_stats: Dict[str, int]) -> List[Dict[str, Any]]:
    path = os.path.join(run_dir, "exchange_rates_agg.filtered.csv")
    if not os.path.isfile(path):
        return []
    with open(path, "r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        rows_raw = list(rdr)

    out: List[Dict[str, Any]] = []
    for r in rows_raw:
        nr = normalize_row_from_csv(r)
        if nr:
            out.append(nr)
        else:
            skip_stats["csv_skipped"] += 1
    return out

def load_from_filtered_jsons(run_dir: str, skip_stats: Dict[str, int]) -> List[Dict[str, Any]]:
    base = os.path.join(run_dir, "exchange_rates_json_filtered")
    if not os.path.isdir(base):
        return []
    out: List[Dict[str, Any]] = []

    for name in os.listdir(base):
        if not name.lower().endswith(".json"):
            continue
        p = os.path.join(base, name)
        fallback_iso = ""
        m = FNAME_DATE_RE.match(name)
        if m:
            yyyy, mm, dd = m.group(1), m.group(2), m.group(3)
            fallback_iso = f"{yyyy}-{mm}-{dd}"

        try:
            with open(p, "r", encoding="utf-8") as f:
                obj = json.load(f)
        except Exception:
            skip_stats["bad_json"] += 1
            continue

        if isinstance(obj, dict) and isinstance(obj.get("data"), list):
            iterable = obj["data"]
        elif isinstance(obj, list):
            iterable = obj
        elif isinstance(obj, dict):
            iterable = [obj]
        else:
            skip_stats["unknown_shape"] += 1
            continue

        for item in iterable:
            if isinstance(item, dict):
                nr = normalize_row_from_json(item, fallback_iso_date=fallback_iso)
                if nr:
                    out.append(nr)
                else:
                    skip_stats["json_row_skipped"] += 1
            else:
                skip_stats["non_dict_row"] += 1
    return out

def dedupe_sort(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    # Deduplicate by (From, To, ValidFrom, Type, Quotation); keep last seen
    seen: Dict[Tuple[str, str, str, str, str], Dict[str, Any]] = {}
    for r in rows:
        key = (r["FromCurrency"], r["ToCurrency"], r["ValidFrom"], r["ExchangeRateType"], r["Quotation"])
        seen[key] = r
    uniq = list(seen.values())

    # Sort by "DD.MM.YYYY": convert to YYYYMMDD for sorting
    def _key(r: Dict[str, Any]) -> Tuple[str, str, str]:
        m = re.fullmatch(r"(\d{2})\.(\d{2})\.(\d{4})", r["ValidFrom"])
        if m:
            dd, mm, yyyy = m.group(1), m.group(2), m.group(3)
            ymd = f"{yyyy}{mm}{dd}"
        else:
            ymd = "00000000"
        return (ymd, r["FromCurrency"], r["ToCurrency"])

    uniq.sort(key=_key)
    return uniq

# ---------- main ----------
def build_payloads_for_all_days() -> List[str]:
    written_paths: List[str] = []
    day_dirs = list_all_day_dirs(BASE_DIR)
    if not day_dirs:
        print(f"No day directories under {BASE_DIR} matching YYYY-MM-DD")
        return written_paths

    for run_dir in day_dirs:
        print(f"[run-dir] {run_dir}")
        skip_stats = {
            "csv_skipped": 0,
            "bad_json": 0,
            "unknown_shape": 0,
            "json_row_skipped": 0,
            "non_dict_row": 0,
        }

        rows = load_from_filtered_csv(run_dir, skip_stats)
        source = "CSV"
        if not rows:
            rows = load_from_filtered_jsons(run_dir, skip_stats)
            source = "JSON"

        rows = dedupe_sort(rows)
        print(f"[source] {source}, rows={len(rows)}; skips={skip_stats}")

        out_path = os.path.join(run_dir, "exchange_rates_payload.json")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(rows, f, ensure_ascii=False, indent=2)
        print(f"[written] {out_path}")
        written_paths.append(out_path)

    print(f"[summary] payloads written: {len(written_paths)}")
    return written_paths

# === RUN ===
if __name__ == "__main__":
    build_payloads_for_all_days()
